# Die Webseite der TH Nürnberg Scrapen

### Einleitung

Die Webseite der TH-Nürnberg wirkt als Ausgangspunkt für die Wissensgrundlage des Chatbots.

In [ ]:
from "webdriver-manager" import ChromeDriverManager

In [ ]:
from bs4 import BeautifulSoup
from bs4.element import Comment
import glob
import json
import sys
import tqdm

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By


### Ausganslage

Als Ausganspunkt für unsere Daten nutzen wir die Home Webseite der TH-Nürnberg. (https://www.th-nuernberg.de/)
Diese Website downloaden wir und suchen alle Links auf andere Webseiten und speichern diese Links in eine Liste.
Als nächsten Schritt rufen wir alle Links aus dieser Liste Auf und sammeln wiederum alle Links von jeder dieser Webseiten.
In der daraus resultierenden Liste sortieren wir alle Links aus, die nicht auf die Webseite der TH verweisen.
Dann laden wir alle Dokumente herunter und speichern sie in der Datenbank.

Zunächst laden Wir die Website

In [ ]:
def get_links():
    print(sys.path)

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get("https://www.google.com")
    links = driver.find_elements(By.CSS_SELECTOR, "a")
    print(links)


In [ ]:

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def get_content(file):
    soup = BeautifulSoup(file,"lxml")

    filename = file.name
    title = soup.find("title")
    if title:
        title = title.text
    else:
        title = ""



    main = soup.find("main")
    portal = soup.find("div", {'class': 'portal'})
    page_container = soup.find("div", {'class': 'page-wrap'})
    
    visible_texts = ""
    if main:
        # print("main found")
        container = main.find("div" ,{'class': 'container'}, recursive=False)
        if container:
            texts = container.find_all(text=True)
            visible_texts = filter(tag_visible, texts)
        # else:
        #     print("no container") 

    elif portal:
        # print(f"portal found {filename}")
        texts = portal.find_all(text=True)
        visible_texts = filter(tag_visible, texts)
        # print(u" ".join(t.strip() for t in visible_texts))

    elif page_container:
        # print(f"page_container found {filename}")
        container = page_container.find("div" ,{'class': 'container'}, recursive=False)
        # container = page_container.children[2]
        if container:
            texts = container.find_all(text=True)
            visible_texts = filter(tag_visible, texts)
        # else:
        #     print("no container") 
        # print(u" ".join(t.strip() for t in visible_texts))

    # else:
    #     if ".pdf" not in filename and ".xml" not in filename:
            # print(f"nothing found {filename}")

        
    # print(u" ".join(t.strip() for t in visible_texts))

    return dict(
        title=title,
        text=u" ".join(t.strip() for t in visible_texts),
        filename=filename
    ) 

def create_video_parameters_json(path, outfile):
    html_files = glob.glob(path)

    # Ausgabe-Datei
    filename = outfile
    videosParameter = []

    with open(filename, "w", encoding="utf-8") as outfile:
        for html_file in tqdm.tqdm(html_files):
            
            with open(html_file, "r", encoding="utf-8") as file:
                data = get_content(file)
                videosParameter.append(data)
        if videosParameter:
            # print(json.dumps(videosParameter))

            outfile.write(json.dumps(videosParameter))

In [ ]:
create_video_parameters_json("htmlfiles/*", "html_data.json")

In [ ]:
import pandas as pd
import sqlite3

json_file = 'html_data.json'
videos_df = pd.read_json(json_file)


database = 'html.sqlite'

with sqlite3.connect(database) as con:
    videos_df.to_sql('html_attrs', con, index=False, if_exists='replace')

Filter only german artciles for better embeddings.

In [ ]:
from db_init import db_get_df, db_save_df

df = db_get_df("html_attrs", ["filename", "title", "text"])

df_en = df[df['filename'].str.contains('file_en')]
df_de = df[~df['filename'].str.contains('file_en')]
db_save_df(df_de,"html_attrs_de")
db_save_df(df_en,"html_attrs_en")

